# Segmenting and Clustering Neighborhoods in Toronto

## Question 1

In [1]:
import pandas as pd
# For scraping the above table, simply use pandas to read the table into a pandas dataframe.

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', attrs={'class': 'wikitable sortable'})

df = dfs[0]

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [2]:
df = df[df.Borough != 'Not assigned']


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [3]:

for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

print("dataframe shape: ", df.shape)

df.head(5)

dataframe shape:  (210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Question 2

In [4]:
cData = pd.read_csv("https://cocl.us/Geospatial_data") 

ndf = df.merge(cData, left_on='Postcode', right_on='Postal Code', how='inner')
ndf.head()


,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763


## Question 3

In [5]:
#toronto_data = ndf[ndf['Borough'].find('Toronto') != -1].reset_index(drop=True)
#toronto_data.head()
#print(toronto_data.shape)

for index, row in ndf.iterrows():
    if row['Borough'].find('Toronto') == -1:
         ndf.drop(index, inplace=True)
        
print(ndf.shape)        
ndf.head()


(74, 6)


,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
6,M9A,Downtown Toronto,Queen's Park,M9A,43.667856,-79.532242
12,M5B,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
13,M5B,Downtown Toronto,Garden District,M5B,43.657162,-79.378937
26,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418


Let's get the geographical coordinates of Toronto.

In [6]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [7]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
# create map of tt using latitude and longitude values
map_tt = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(ndf['Latitude'], ndf['Longitude'], ndf['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tt)  
    
map_tt

Start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [9]:
CLIENT_ID = 'DLN0USNHQV1V0ESSXGF2SRKC4VC1RLYBCAKYIW2OIQ4TO4JJ' # your Foursquare ID
CLIENT_SECRET = 'KQYVJCZIEA0WV5KLSJN4CNPGGLZI3YZD4YCLXPRBBUBFQS01' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

ndf.loc[2, 'Neighbourhood']

Your credentails:
CLIENT_ID: DLN0USNHQV1V0ESSXGF2SRKC4VC1RLYBCAKYIW2OIQ4TO4JJ
CLIENT_SECRET:KQYVJCZIEA0WV5KLSJN4CNPGGLZI3YZD4YCLXPRBBUBFQS01


'Harbourfront'

Create a function to repeatedly get the venue info of all the neighborhoods 

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
tt_venues = getNearbyVenues(names=ndf['Neighbourhood'],
                                   latitudes=ndf['Latitude'],
                                   longitudes=ndf['Longitude']
                                  )

print(tt_venues.shape)
tt_venues.head()

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [12]:
tt_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,16,16,16,16,16,16
Berczy Park,56,56,56,56,56,56
Brockton,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
CN Tower,16,16,16,16,16,16
Cabbagetown,43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
Chinatown,94,94,94,94,94,94


Let's find out how many unique categories can be curated from all the returned venues

In [13]:
print('There are {} uniques categories.'.format(len(tt_venues['Venue Category'].unique())))

There are 231 uniques categories.


### Analyze Each Neighborhood

In [14]:
# one hot encoding
tt_onehot = pd.get_dummies(tt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tt_onehot['Neighbourhood'] = tt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tt_onehot.columns[-1]] + list(tt_onehot.columns[:-1])
tt_onehot = tt_onehot[fixed_columns]

tt_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [15]:
tt_grouped = tt_onehot.groupby('Neighbourhood').mean().reset_index()
tt_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Bathurst Quay,0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667
5,CN Tower,0.000000,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cabbagetown,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.011905,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
8,Chinatown,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.010638,0.00000,0.00000,0.00,0.031915,0.000000,0.042553,0.010638,0.000000,0.000000
9,Christie,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [16]:
num_top_venues = 5

for hood in tt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tt_grouped[tt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.05
2        Steakhouse  0.04
3   Thai Restaurant  0.04
4  Asian Restaurant  0.03


----Bathurst Quay----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4               Bar  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1            Beer Bar  0.04
2        Cocktail Bar  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton----
                   venue  freq
0         Breakfast Spot  0.09
1            Coffee Shop  0.09
2                   Café  0.09
3                 Bakery  0.05
4  Performing Arts Venue  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.13
1           Yoga Studio  0.07
2         Auto Workshop  0.07
3         Garden Center  0.07
4  Fast Food Restaurant  0.07


----CN Tow

Let's write a function to sort the venues in descending order.

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [18]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tt_grouped['Neighbourhood']

for ind in np.arange(tt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bar,Bakery,Salad Place,Sushi Restaurant,Asian Restaurant
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Plane,Coffee Shop,Bar,Boat or Ferry,Sculpture Garden
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Bakery,Steakhouse,Beer Bar,Farmers Market,Beach
3,Brockton,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Bakery,Nightclub,Pet Store,Climbing Gym,Restaurant,Burrito Place
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place,Restaurant


## clustering
Run k-means to cluster the neighborhood into 5 clusters.

In [19]:
# set number of clusters
kclusters = 5

tt_grouped_clustering = tt_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 1, 1, 3, 1, 1, 1, 1], dtype=int32)

 create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tt_merged = ndf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tt_merged = tt_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='inner')

tt_merged.head(20) # check the last columns!

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Café,Antique Shop,Greek Restaurant
12,M5B,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Tea Room,Bakery,Ramen Restaurant,Diner
13,M5B,Downtown Toronto,Garden District,M5B,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Middle Eastern Restaurant,Tea Room,Bakery,Ramen Restaurant,Diner
26,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Breakfast Spot,Clothing Store,Hotel,Diner,Park,Pizza Place
189,M4X,Downtown Toronto,St. James Town,M4X,43.667967,-79.367675,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Breakfast Spot,Clothing Store,Hotel,Diner,Park,Pizza Place
35,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,1,Health Food Store,Coffee Shop,Neighborhood,Trail,Pub,Yoga Studio,Doner Restaurant,Diner,Discount Store,Dog Run
36,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Bakery,Steakhouse,Beer Bar,Farmers Market,Beach
40,M5G,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Burger Joint,Café,Sandwich Place,Ice Cream Shop,Chinese Restaurant,Bubble Tea Shop,Bar,Spa
41,M6G,Downtown Toronto,Christie,M6G,43.669542,-79.422564,1,Grocery Store,Café,Park,Diner,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Nightclub
48,M5H,Downtown Toronto,Adelaide,M5H,43.650571,-79.384568,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bar,Bakery,Salad Place,Sushi Restaurant,Asian Restaurant


Finally, let's visualize the resulting clusters

In [21]:
#tt_merged = tt_merged['Cluster Labels' != float('NaN')]

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tt_merged['Latitude'], tt_merged['Longitude'], tt_merged['Neighbourhood'], tt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters